Library


In [33]:
import pickle
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

Main Path


In [34]:
PATH_MAIN = r"C:\Users\Lulay\Documents\GitHub\Dasar-Kecerdasan-Artificial_Tugas-Besar"
# PATH_MAIN = r"D:\Kuliah\Matkul\Semester 4\DASAR KECERDASAN ARTIFICIAL (DKA)\[2] Tugas\[3] Tugas Besar\Dasar-Kecerdasan-Artificial_Tugas-Besar"

PATH_DATASET = f"{PATH_MAIN}/Dataset"

Konstanta


In [35]:
SAMPLE_RATE = 48000
BATCH_SIZE = 32
SEGMENT_DURATION = int(1 * SAMPLE_RATE)
OVERLAP_DURATION = int(0.5 * SAMPLE_RATE)
RANDOM_STATE = 21
BUFFER_SIZE = 1000

Random Seed


In [36]:
RANDOM_SEED = 21

try:
    from sklearn.utils import check_random_state
    random_state = check_random_state(RANDOM_SEED)
except ImportError:
    pass

# Dataset


Load Dataset Raw


In [37]:
with open(f"{PATH_DATASET}/dataset.pkl", "rb") as f:
    dataset = pickle.load(f)

In [38]:
len(dataset)

1894

In [39]:
dataset[0]

{'nama': 'C:\\Users\\Lulay\\Documents\\GitHub\\Dasar-Kecerdasan-Artificial_Tugas-Besar\\Dataset\\xeno-canto\\19655.mp3',
 'timestamp': [0.0, 1.0],
 'label': 2,
 'raw': array([-6.6490856e-13, -2.0503420e-13,  1.9726230e-12, ...,
         1.7010156e-02,  1.5655354e-02,  1.3218733e-02], dtype=float32),
 'rms': 0.011059444,
 'zcr': 0.23255173703457446,
 'mfcc': array([[-9.2709552e+02, -9.2709552e+02, -8.6692578e+02, ...,
         -4.9831238e+02, -4.7504291e+02, -3.9764847e+02],
        [ 0.0000000e+00,  0.0000000e+00, -1.4798780e+01, ...,
         -8.1901199e+01, -5.0254303e+01,  2.3999417e+00],
        [ 0.0000000e+00,  0.0000000e+00, -5.1890976e+01, ...,
         -2.6562943e+02, -2.3930214e+02, -1.4375620e+02],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  8.7950077e+00, ...,
          7.3784947e-01,  1.3385715e+00,  2.1685948e+00],
        [ 0.0000000e+00,  0.0000000e+00, -1.2696293e+01, ...,
          5.2022276e+00,  3.3751645e+00, -6.7336679e+00],
        [ 0.0000000e+00,  0.

`Label 1 untuk suara non-burung dan 0 untuk suara burung`


Assign variabel X dan y


In [40]:
X = [[data["maxamp"], data["mfcc_std"]] for data in dataset]
original_labels = [int(data["label"]) for data in dataset]
y = [1 if label == 0 or label == 1 else 2 for label in original_labels]
# y = [int(data["label"]) for data in dataset]

Check


In [41]:
pd.Series(y).value_counts()

2    1068
1     826
Name: count, dtype: int64

In [42]:
print("Data:\n", X[0])
print("Label: ", y[0])

Data:
 [0.08531428, 137.34595]
Label:  2


Split


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=RANDOM_STATE, stratify=y_test)

Normalisasi


In [44]:
scl = StandardScaler()
X_train_scaled = scl.fit_transform(X_train)
X_test_scaled = scl.transform(X_test)
X_val_scaled = scl.transform(X_val)

Save scaler


In [45]:
joblib.dump(scl, f'{PATH_MAIN}/model/p_scaler.pkl')

['C:\\Users\\Lulay\\Documents\\GitHub\\Dasar-Kecerdasan-Artificial_Tugas-Besar/model/p_scaler.pkl']

Check


In [46]:
display(
    X_train_scaled.shape,
    X_test_scaled.shape,
    X_val_scaled.shape
)

(1515, 2)

(189, 2)

(190, 2)

# Train


Model yang akan digunakan untuk klasifikasi suara burung:

1. Fuzzy Mamdani
2. Fuzzy Sugeno
3. Logistic Regression
4. Descision Tree
5. LGBM Classifier


## Fuzzy Mamdani


In [47]:
# def trapmf(x, a, b, c, d):
#     return np.maximum(np.minimum(np.minimum((x - a)/(b - a + 1e-9), 1), (d - x)/(d - c + 1e-9)), 0)

# class FuzzyVariable:
#     def __init__(self, name, range_):
#         self.name = name
#         self.range = range_
#         self.sets = {}
#     def add_trapezoidal(self, set_name, a, b, c, d):
#         self.sets[set_name] = lambda x, a=a, b=b, c=c, d=d: trapmf(x, a, b, c, d)
#     def fuzzify(self, value):
#         return {name: mf(value) for name, mf in self.sets.items()}

# class FuzzySystem:
#     def __init__(self):
#         self.inputs = {}
#         self.outputs = {}
#         self.rules = []
#         self.input_values = {}
#     def add_input(self, fuzzy_var):
#         self.inputs[fuzzy_var.name] = fuzzy_var
#     def add_output(self, fuzzy_var):
#         self.outputs[fuzzy_var.name] = fuzzy_var
#     def add_rule(self, antecedents, consequents):
#         self.rules.append((antecedents, consequents))
#     def set_input(self, var, value):
#         self.input_values[var] = value
#     def evaluate(self):
#         agg_output = {name: np.zeros(1000) for name in self.outputs}
#         x_values = {name: np.linspace(v.range[0], v.range[1], 1000) for name, v in self.outputs.items()}
#         for antecedents, consequents in self.rules:
#             strengths = []
#             for var, set_name in antecedents:
#                 mf_val = self.inputs[var].sets[set_name](self.input_values[var])
#                 strengths.append(mf_val)
#             rule_strength = min(strengths)
#             for var, set_name in consequents:
#                 mf = self.outputs[var].sets[set_name](x_values[var])
#                 agg_output[var] = np.maximum(agg_output[var], np.minimum(rule_strength, mf))
#         crisp = {}
#         for var, vals in agg_output.items():
#             x = x_values[var]
#             if np.sum(vals) == 0:
#                 crisp[var] = 0
#             else:
#                 crisp[var] = np.sum(x * vals) / np.sum(vals)
#         return crisp

In [48]:
# specification = {
#     "input_vars": {
#         "rms": {
#             "range": [0, 0.445492],
#             "sets": {
#                 "low":    [0.0, 0.0, 0.01, 0.03],
#                 "medium": [0.02, 0.04, 0.12, 0.2],
#                 "high":   [0.15, 0.25, 0.445492, 0.445492],
#             }
#         },
#         "zcr": {
#             "range": [0, 0.363307],
#             "sets": {
#                 "infrequent": [0.0, 0.0, 0.01, 0.02],
#                 "moderate":   [0.015, 0.025, 0.08, 0.15],
#                 "frequent":   [0.08, 0.15, 0.363307, 0.363307]
#             }
#         }
#     },
#     "output_var": {
#         "loudness": {
#             "range": [0, 1],
#             "sets": {
#                 "improbable": [0.0, 0.0, 0.2, 0.4],
#                 "probable":   [0.2, 0.4, 0.6, 0.8],
#                 "certain":    [0.6, 0.8, 1.0, 1.0]
#             }
#         }
#     },
#     "rules": [
#         {"if": {"rms": "low",    "zcr": "infrequent"}, "then": {"loudness": "improbable"}},
#         {"if": {"rms": "low",    "zcr": "moderate"},   "then": {"loudness": "improbable"}},
#         {"if": {"rms": "low",    "zcr": "frequent"},   "then": {"loudness": "probable"}},
#         {"if": {"rms": "medium", "zcr": "infrequent"}, "then": {"loudness": "improbable"}},
#         {"if": {"rms": "medium", "zcr": "moderate"},   "then": {"loudness": "probable"}},
#         {"if": {"rms": "medium", "zcr": "frequent"},   "then": {"loudness": "probable"}},
#         {"if": {"rms": "high",   "zcr": "infrequent"}, "then": {"loudness": "probable"}},
#         {"if": {"rms": "high",   "zcr": "moderate"},   "then": {"loudness": "probable"}},
#         {"if": {"rms": "high",   "zcr": "frequent"},   "then": {"loudness": "certain"}}
#     ]
# }

# fs = FuzzySystem()

# for name, info in specification["input_vars"].items():
#     var = FuzzyVariable(name, info["range"])
#     [var.add_trapezoidal(n, *pts) for n, pts in info["sets"].items()]
#     fs.add_input(var)

# for name, info in specification["output_var"].items():
#     var = FuzzyVariable(name, info["range"])
#     [var.add_trapezoidal(n, *pts) for n, pts in info["sets"].items()]
#     fs.add_output(var)

# for rule in specification["rules"]:
#     fs.add_rule(
#         [(k, v) for k, v in rule["if"].items()],
#         [(k, v) for k, v in rule["then"].items()]
#     )

# def is_loud(rms_val, zcr_val):
#     def fuzzify(val, sets):
#         return {
#             lbl: (
#                 0.0 if val <= a or val >= d else
#                 (val - a) / (b - a) if a < val <= b and b != a else
#                 1.0 if b < val <= c else
#                 (d - val) / (d - c) if c < val < d and d != c else 0.0
#             )
#             for lbl, (a, b, c, d) in sets.items()
#         }

#     rms_d = fuzzify(rms_val, specification["input_vars"]["rms"]["sets"])
#     zcr_d = fuzzify(zcr_val, specification["input_vars"]["zcr"]["sets"])
#     scores = {"improbable": 0.0, "probable": 0.0, "certain": 0.0}

#     for r in specification["rules"]:
#         deg = min(rms_d[r["if"]["rms"]], zcr_d[r["if"]["zcr"]])
#         out = r["then"]["loudness"]
#         scores[out] = max(scores[out], deg)

#     return 1 if max(scores, key=scores.get) in ["probable", "certain"] else 2

In [49]:
# y_pred = [is_loud(rms, zcr) for rms, zcr in X_val_scaled]

In [50]:
# print("Validasi")
# print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
# print(f"Precision: {precision_score(y_val, y_pred)}")
# print(f"Recall: {recall_score(y_val, y_pred)}")
# print(f"F1 Score: {f1_score(y_val, y_pred)}")
# print(f"Confusion Matrix:\n{confusion_matrix(y_val, y_pred)}")

## Fuzzy Sugeno


## Logistic Regression


Inisialisasi


In [51]:
lr_model = LogisticRegression(random_state=RANDOM_STATE)

training


In [52]:
lr_model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=21)

Evaluasi


In [53]:
y_pred = lr_model.predict(X_val_scaled)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

Validasi
Accuracy: 0.7
Precision: 0.6805555555555556
Recall: 0.5903614457831325
F1 Score: 0.632258064516129
Confusion Matrix:
 [[49 34]
 [23 84]]


In [54]:
y_pred = lr_model.predict(X_test_scaled)

print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Test
Accuracy: 0.7883597883597884
Precision: 0.8
Recall: 0.6829268292682927
F1 Score: 0.7368421052631579
Confusion Matrix:
 [[56 26]
 [14 93]]


## Descision Tree


Inisialisasi


In [55]:
dt_model = DecisionTreeClassifier(random_state=RANDOM_STATE)

training


In [56]:
dt_model.fit(X_train_scaled, y_train)

DecisionTreeClassifier(random_state=21)

Evaluasi


In [57]:
y_pred = dt_model.predict(X_val_scaled)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

Validasi
Accuracy: 0.7736842105263158
Precision: 0.7272727272727273
Recall: 0.7710843373493976
F1 Score: 0.7485380116959064
Confusion Matrix:
 [[64 19]
 [24 83]]


In [58]:
y_pred = dt_model.predict(X_test_scaled)

print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Test
Accuracy: 0.7407407407407407
Precision: 0.7037037037037037
Recall: 0.6951219512195121
F1 Score: 0.6993865030674846
Confusion Matrix:
 [[57 25]
 [24 83]]


## LGBM


Inisialisasi


In [59]:
lgbm_model = LGBMClassifier(random_state=RANDOM_STATE, verbose=-1)

training


In [60]:
lgbm_model.fit(X_train_scaled, y_train)

LGBMClassifier(random_state=21, verbose=-1)

Evaluasi


In [61]:
y_pred = lgbm_model.predict(X_val_scaled)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

Validasi
Accuracy: 0.7842105263157895
Precision: 0.7333333333333333
Recall: 0.7951807228915663
F1 Score: 0.7630057803468208
Confusion Matrix:
 [[66 17]
 [24 83]]


In [62]:
y_pred = lgbm_model.predict(X_test_scaled)

print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Test
Accuracy: 0.8095238095238095
Precision: 0.7804878048780488
Recall: 0.7804878048780488
F1 Score: 0.7804878048780488
Confusion Matrix:
 [[64 18]
 [18 89]]


Save


In [63]:
joblib.dump(lgbm_model, f'{PATH_MAIN}/model/p_lgbm.pkl')

['C:\\Users\\Lulay\\Documents\\GitHub\\Dasar-Kecerdasan-Artificial_Tugas-Besar/model/p_lgbm.pkl']